In [9]:
%load_ext autoreload
%autoreload 2

import torch
import pyterrier as pt
import pandas as pd

from src.neural_ranker.ranker import NeuralRanker
from src.neural_ranker.produce_rankings import IRDataset, Processor
from src.llm.llm import LLM_zeroshot, LLM_query_exp
from domain_adaptation import self_training_domain_adaptation
from main import rank_with_base_model, contrastive_train_neural_ranker, rank_with_contrastive_model, pseudo_labels_fine_tune, rank_with_pseudo_labels_model
from eval import Model, evaluate_rankings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    gpu_mem_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    gpu_mem_reserved = torch.cuda.memory_reserved(0) / 1024**3
    gpu_mem_allocated = torch.cuda.memory_allocated(0) / 1024**3
    gpu_mem_free = gpu_mem_total - gpu_mem_reserved
    print(f"GPU Memory: Total: {gpu_mem_total:.2f}GB, Reserved: {gpu_mem_reserved:.2f}GB, "
        f"Allocated: {gpu_mem_allocated:.2f}GB, Free: {gpu_mem_free:.2f}GB")

Using device: cuda
GPU Memory: Total: 15.99GB, Reserved: 0.00GB, Allocated: 0.00GB, Free: 15.99GB


In [3]:
bair = 'irds:beir/trec-covid'
cord19 = 'irds:cord19/trec-covid'

bair_dataset = IRDataset(bair, max_docs=None)
cord19_dataset = IRDataset(cord19, max_docs=None)

Loading up to None documents from irds:beir/trec-covid...


beir/trec-covid documents: 100%|██████████| 171332/171332 [00:00<00:00, 302268.09it/s]
Loading Documents: 171332it [00:00, 302267.07it/s]


Loading up to None documents from irds:cord19/trec-covid...


cord19/trec-covid documents: 100%|██████████| 192509/192509 [00:00<00:00, 310197.61it/s]
Loading Documents: 192509it [00:00, 310197.61it/s]


In [4]:
rank_with_base_model(cord19_dataset, device)

Checking for existing embeddings...
Loaded 192509 existing document embeddings.
Document encoding complete. Now ranking queries...
There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
Loaded 50 queries
Ranking 50 queries against 192509 documents...


Ranking Queries: 100%|██████████| 50/50 [00:27<00:00,  1.85it/s]


In [6]:
contrastive_train_neural_ranker(device=device)

Loading dataset...
Loading up to None documents from irds:beir/trec-covid...


beir/trec-covid documents: 100%|██████████| 171332/171332 [00:00<00:00, 318056.10it/s]
Loading Documents: 171332it [00:00, 318647.87it/s]


Processing documents...


Processing documents: 100%|██████████| 171332/171332 [00:00<00:00, 1146138.45it/s]


Loaded 171332 documents for contrastive training.

Initializing augmentor...
Creating contrastive dataset...

Creating dataloader...

Dataloader configuration: batch_size=8, num_workers=0

Initializing model...
Model has 109,482,240 trainable parameters

Initializing trainer...

Training for up to 3 epochs with early stopping (patience=2, min_delta=0.001)...


Epoch 1/3:   1%|▏         | 318/21417 [01:29<1:39:02,  3.55it/s, loss=0.0010]

Loss 0.000999 is below threshold. Stopping early.

Saving model to models/contrastive_model.pt...


Model saved successfully!


In [13]:
rank_with_contrastive_model(cord19_dataset, device)

c:\Users\bogda\Desktop\TU Delft\Q3\DSAIT4050-IR\research_project\ranker_ir\main.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ranker.load_state_dict(torch.load("mode

Checking for existing embeddings...
No existing embeddings found or file corrupted. Starting from scratch.

Processing chunk 1/39 (documents 0 to 4999)...


Encoding Chunk 1: 100%|██████████| 79/79 [00:24<00:00,  3.20it/s]



Processing chunk 2/39 (documents 5000 to 9999)...


Encoding Chunk 2: 100%|██████████| 79/79 [00:23<00:00,  3.39it/s]


...

Encoding Chunk 38: 100%|██████████| 79/79 [00:24<00:00,  3.21it/s]



Processing chunk 39/39 (documents 190000 to 192508)...


Encoding Chunk 39: 100%|██████████| 40/40 [00:12<00:00,  3.25it/s]


Completed processing 192509 documents.
Document encoding complete. Now ranking queries...
There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
Loaded 50 queries
Ranking 50 queries against 192509 documents...


Ranking Queries: 100%|██████████| 50/50 [00:32<00:00,  1.53it/s]


In [11]:
evaluate_rankings(model=Model.BASE, dataset_name=cord19)

Evaluating rankings with metrics@10...
Loaded 9625450 ranking entries

=== Evaluation Results ===
map: 0.1859
ndcg: 0.6866
ndcg_cut_10: 0.6570
P_10: 0.6880
recall_10: 0.0165
recip_rank: 0.8403

Evaluation results saved to evaluation_results/base\evaluation_results.csv

Per-query evaluation results:
Query 1:
    map: 0.1281
    recip_rank: 1.0000
    P_10: 0.5000
    recall_10: 0.0072
    ndcg: 0.6776
    ndcg_cut_10: 0.4959
-------------------------

Query 2:
    map: 0.0585
    recip_rank: 0.5000
    P_10: 0.6000
    recall_10: 0.0179
    ndcg: 0.5997
    ndcg_cut_10: 0.5527
-------------------------

Query 3:
    map: 0.1977
    recip_rank: 1.0000
    P_10: 0.9000
    recall_10: 0.0138
    ndcg: 0.7465
    ndcg_cut_10: 0.9364
-------------------------

Query 4:
    map: 0.0535
    recip_rank: 0.1000
    P_10: 0.1000
    recall_10: 0.0018
    ndcg: 0.5830
    ndcg_cut_10: 0.0636
-------------------------

Query 5:
    map: 0.1349
    recip_rank: 0.3333
    P_10: 0.4000
    recall_10: 

{'map': 0.18585182567772524,
 'ndcg': 0.6865586940416332,
 'ndcg_cut_10': 0.6569951008404138,
 'P_10': 0.6879999999999998,
 'recall_10': 0.01649486349187759,
 'recip_rank': 0.840343137254902}

In [14]:
evaluate_rankings(model=Model.CONTRASTIVE, dataset_name=cord19)

Evaluating rankings with metrics@10...
Loaded 9625450 ranking entries

=== Evaluation Results ===
map: 0.0033
ndcg: 0.4235
ndcg_cut_10: 0.0000
P_10: 0.0000
recall_10: 0.0000
recip_rank: 0.0026

Evaluation results saved to evaluation_results/contrastive\evaluation_results.csv

Per-query evaluation results:
Query 1:
    map: 0.0046
    recip_rank: 0.0021
    P_10: 0.0000
    recall_10: 0.0000
    ndcg: 0.4614
    ndcg_cut_10: 0.0000
-------------------------

Query 2:
    map: 0.0060
    recip_rank: 0.0078
    P_10: 0.0000
    recall_10: 0.0000
    ndcg: 0.4451
    ndcg_cut_10: 0.0000
-------------------------

Query 3:
    map: 0.0031
    recip_rank: 0.0004
    P_10: 0.0000
    recall_10: 0.0000
    ndcg: 0.4277
    ndcg_cut_10: 0.0000
-------------------------

Query 4:
    map: 0.0031
    recip_rank: 0.0022
    P_10: 0.0000
    recall_10: 0.0000
    ndcg: 0.4253
    ndcg_cut_10: 0.0000
-------------------------

Query 5:
    map: 0.0036
    recip_rank: 0.0002
    P_10: 0.0000
    reca

{'map': 0.003338411312322189,
 'ndcg': 0.42352346833935955,
 'ndcg_cut_10': 0.0,
 'P_10': 0.0,
 'recall_10': 0.0,
 'recip_rank': 0.002590893580737511}